In [7]:
import pandas as pd
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, Input
import os
import datetime
from tensorflow.keras.applications import ResNet50
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.utils import to_categorical

# data loading 


In [21]:
img_size = 32
num_classes = 28 


path = ".././data/"


def load_data(path="data/", num_classes=None):
    try:
        x_train_df = pd.read_csv(f'{path}csvTrainImages 13440x1024.csv', header=None)
        y_train_df = pd.read_csv(f'{path}csvTrainLabel 13440x1.csv', header=None)
        x_test_df = pd.read_csv(f'{path}csvTestImages 3360x1024.csv', header=None)
        y_test_df = pd.read_csv(f'{path}csvTestLabel 3360x1.csv', header=None)
    except FileNotFoundError:
        print("file was not found at", path)
        return (None, None), (None, None)

    x_train = x_train_df.to_numpy()
    y_train = y_train_df.to_numpy()
    x_test = x_test_df.to_numpy()
    y_test = y_test_df.to_numpy()

    x_train = x_train.reshape(-1, 32, 32, 1).astype('float32')
    x_test = x_test.reshape(-1, 32, 32, 1).astype('float32')
    x_train /= 255.0
    x_test /= 255.0

    y_train = y_train.flatten().astype('int32')
    y_test = y_test.flatten().astype('int32')

    print("example before minus", y_train[1])
    y_train = y_train - 1
    y_test = y_test - 1
    print("example after minus", y_train[1])

    if num_classes is None:
        num_classes = int(max(y_train.max(), y_test.max()) + 1)

    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    print(f"data loaded and train shape {x_train.shape} , and test shape is {x_test.shape}")

    return (x_train, y_train), (x_test, y_test)


In [22]:
(x_train, y_train), (x_test, y_test) = load_data(path)

example before minus 1
example after minus 0
data loaded and train shape (13440, 32, 32, 1) , and test shape is (3360, 32, 32, 1)


# VGG-19

<img src="/media/external-drive/projects/deep_learning_project/VGG--19-Architecture-.webp" alt="VGG-19 Architecture" width="700"/>


In [11]:


def build_vgg19(input_shape=(32, 32, 1), num_classes=28):
    
    inputs = Input(shape=input_shape, name="img_input")

    ##block 2##
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    ##block 2 ##
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    ## block 3 ##
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    ## block 4 ##
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    ## Block 5##
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    ## head ##
    x = layers.Flatten(name='flatten')(x)
    
    x = layers.Dense(1024, activation='relu', name='fc1')(x)   ## i used  smallerr forward net since the standart one for vgg is overkill and to train to faster##
    x = layers.Dropout(0.5)(x) # Essential to prevent overfitting
    x = layers.Dense(1024, activation='relu', name='fc2')(x)
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="VGG19_Custom")
    return model




# def build_vgg19_fixed(input_shape=(32, 32, 1), num_classes=28):
#     inputs = Input(shape=input_shape, name="img_input")
    
#     def conv_block(x, filters, repeat):
#         for _ in range(repeat):
#             x = layers.Conv2D(filters, (3, 3), padding='same')(x)
#             x = layers.BatchNormalization()  # <--- THE MAGIC FIX
#             x = layers.Activation('relu')(x)
#         x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
#         return x



#     x = conv_block(inputs, 64, 2)   ## block 2##
#     x = conv_block(x, 128, 2)       ## block 2##
#     x = conv_block(x, 256, 4)       ## block 3##
#     x = conv_block(x, 512, 4)       ## block 4  i removed a block and add normalization since the model was too deep for my data and to train faster##   
    
#     x = layers.Conv2D(512, (3, 3), padding='same')(x)
#     x = layers.BatchNormalization()
#     x = layers.Activation('relu')(x)
#     x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x) 
    
#     x = layers.Flatten()(x)
    
#     x = layers.Dense(1024)(x)
#     x = layers.BatchNormalization()
#     x = layers.Activation('relu')(x)
#     x = layers.Dropout(0.5)(x)
    
#     x = layers.Dense(1024)(x)
#     x = layers.BatchNormalization()
#     x = layers.Activation('relu')(x)
#     x = layers.Dropout(0.5)(x)
    
#     outputs = layers.Dense(num_classes, activation='softmax')(x)

#     model = models.Model(inputs=inputs, outputs=outputs, name="VGG19_BN_Fixed")
#     return model




In [12]:
model_vgg = build_vgg19(input_shape=(32, 32, 1), num_classes=28)
model_vgg.summary()

model_vgg.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model: "VGG19_Custom"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ img_input (InputLayer)          │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             

 Total params: 21,626,844 (82.50 MB)

 Trainable params: 21,626,844 (82.50 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# logs for tranin and call backs and logs

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_dir = "models/"

callbacks_list = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "vgg19_best.keras"),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
]

In [28]:
history_vgg = model_vgg.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=callbacks_list
)

evaluate_model_plotly(model_vgg, x_test, y_test, model_name="VGG19 Custom")

Epoch 1/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.0344 - loss: 3.3335

2025-11-27 22:32:58.667413: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_240_0', 428 bytes spill stores, 524 bytes spill loads

2025-11-27 22:32:59.186090: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_240', 220 bytes spill stores, 220 bytes spill loads

2025-11-27 22:33:01.311312: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_240', 176 bytes spill stores, 392 bytes spill loads

2025-11-27 22:33:01.324427: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_247', 220 bytes spill stores, 220 bytes spill loads

2025-11-27 22:33:02.185750: W external/local_xla/xla/t


Epoch 1: val_accuracy improved from -inf to 0.03571, saving model to models/vgg19_best.keras
210/210 ━━━━━━━━━━━━━━━━━━━━ 59s 214ms/step - accuracy: 0.0344 - loss: 3.3335 - val_accuracy: 0.0357 - val_loss: 3.3327
Epoch 2/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0313 - loss: 3.3333
Epoch 2: val_accuracy did not improve from 0.03571
210/210 ━━━━━━━━━━━━━━━━━━━━ 22s 105ms/step - accuracy: 0.0313 - loss: 3.3333 - val_accuracy: 0.0357 - val_loss: 3.3322
Epoch 3/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0381 - loss: 3.3324
Epoch 3: val_accuracy did not improve from 0.03571
210/210 ━━━━━━━━━━━━━━━━━━━━ 22s 105ms/step - accuracy: 0.0381 - loss: 3.3324 - val_accuracy: 0.0357 - val_loss: 3.3322
Epoch 4/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.0347 - loss: 3.3327
Epoch 4: val_accuracy did not improve from 0.03571
210/210 ━━━━━━━━━━━━━━━━━━━━ 21s 102ms/step - accuracy: 0.0347 - loss: 3.3327 - val_accuracy: 0.0357 - val_loss: 3.3322
Epoch 5/20
2

/home/theodoros/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/theodoros/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/theodoros/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
inputs = Input(shape=(32, 32, 1))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    
model_simple = models.Model(inputs, outputs, name="Simple_CNN")


model_simple.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_simple.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
210/210 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1680 - loss: 2.9976 - val_accuracy: 0.0363 - val_loss: 6.1239
Epoch 2/5
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5509 - loss: 1.3570 - val_accuracy: 0.1265 - val_loss: 3.8232
Epoch 3/5
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7315 - loss: 0.8007 - val_accuracy: 0.8161 - val_loss: 0.5471
Epoch 4/5
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8092 - loss: 0.5537 - val_accuracy: 0.7866 - val_loss: 0.6052
Epoch 5/5
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8724 - loss: 0.3822 - val_accuracy: 0.9232 - val_loss: 0.2614


In [25]:
def evaluate_model_plotly(model, x_test, y_test, model_name="Model"):

    
    # --- 1. Get Predictions ---
    print(f"📊 Calculating Metrics for {model_name}...")
    y_pred_probs = model.predict(x_test, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    
    # Handle One-Hot Encoding
    if y_test.shape[-1] > 1: 
        y_true_classes = np.argmax(y_test, axis=1)
    else:
        y_true_classes = y_test

    # --- 2. Text Report (Precision, Recall, F1) ---
    print(f"\n🔹 Classification Report [{model_name}]:")
    print(classification_report(y_true_classes, y_pred_classes))

    # --- 3. Plotly Confusion Matrix ---
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    
    # Create Labels (0 to 27)
    labels = [str(i) for i in range(28)]
    
    fig_cm = px.imshow(
        cm,
        text_auto=True,
        labels=dict(x="Predicted Label", y="True Label", color="Count"),
        x=labels,
        y=labels,
        color_continuous_scale='Blues',
        title=f"Confusion Matrix: {model_name}"
    )
    fig_cm.update_layout(
        width=800, height=800,
        title_x=0.5,
        font=dict(size=14)
    )
    fig_cm.show()

    # --- 4. Plotly ROC Curve (Macro Average) ---
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes = y_test.shape[1]
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute Macro-Average ROC
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes

    # Create Plotly Figure
    fig_roc = go.Figure()

    # Add the Macro Average Line
    fig_roc.add_trace(go.Scatter(
        x=all_fpr, y=mean_tpr,
        mode='lines',
        name=f'Macro-average ROC (AUC = {auc(all_fpr, mean_tpr):.3f})',
        line=dict(color='navy', width=4, dash='solid')
    ))

    # Add the Diagonal "Random Guess" Line
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='Random Chance',
        line=dict(color='gray', width=2, dash='dash')
    ))

    fig_roc.update_layout(
        title=f"ROC Curve: {model_name}",
        title_x=0.5,
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800, height=600,
        legend=dict(x=0.7, y=0.1)
    )
    fig_roc.show()

# Residual Networks (ResNet) - Deep Learning
ResNet, which was proposed in 2015 by researchers at Microsoft Research introduced a new architecture called Residual Network. 
In order to address the vanishing/exploding gradient problem, ResNet uses skip (shortcut) connections that let gradients flow directly through identity mappings.

![ResNet architecture](/media/external-drive/projects/deep_learning_project/ResNet.PNG)

*Figure 1 — ResNet architecture (/media/external-drive/projects/deep_learning_project/ResNet.PNG)*

![Residual block](/media/external-drive/projects/deep_learning_project/Residual-Block.PNG)

*Figure 2 — Residual block (shortcut connection) (/media/external-drive/projects/deep_learning_project/Residual-Block.PNG)*


In [23]:

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

inputs = Input(shape=(32, 32, 1), name="img_input")

## original model was trained on rgb image so we need to convert 1 channel to 3 channels##
x = layers.Conv2D(3, (3, 3), padding='same', name='rgb_adapter')(inputs)

# We initialize ResNet with a standard input_shape to avoid the "Layer count mismatch" error
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) ## include_top=Fals We cut off the original classification head##

base_model.trainable = False ##freezes the orignal model##

# Pass the adapter output 'x' into the base_model here
x = base_model(x, training=False)

## new classifcation head the trainable part##
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(28, activation='softmax')(x) # 28 Arabic Classes

model_resnet = models.Model(inputs=inputs, outputs=outputs, name="ResNet50_Transfer")

model_resnet.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_resnet.summary()

history_resnet = model_resnet.fit(
    x_train, y_train,
    epochs=15,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback]
)

Model: "ResNet50_Transfer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ img_input (InputLayer)          │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rgb_adapter (Conv2D)            │ (None, 32, 32, 3)      │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 1, 1, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 28)             │        14,364 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,651,194 (94.04 MB)

 Trainable params: 1,063,482 (4.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

...

  1/210 ━━━━━━━━━━━━━━━━━━━━ 1:06:13 19s/step - accuracy: 0.0156 - loss: 5.0653

I0000 00:00:1764274816.012603   16061 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


208/210 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0384 - loss: 3.5764

2025-11-27 22:20:27.877837: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1717_0', 112 bytes spill stores, 224 bytes spill loads

2025-11-27 22:20:28.190287: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1717_0', 204 bytes spill stores, 204 bytes spill loads

2025-11-27 22:20:28.501237: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1717', 220 bytes spill stores, 576 bytes spill loads

2025-11-27 22:20:32.318267: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 141557760 exceeds 10% of free system memory.
2025-11-27 22:20:32.451683: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 125829120 exceeds 10% of f

210/210 ━━━━━━━━━━━━━━━━━━━━ 40s 101ms/step - accuracy: 0.0384 - loss: 3.5736 - val_accuracy: 0.0780 - val_loss: 3.2839
Epoch 2/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.0576 - loss: 3.2486 - val_accuracy: 0.1170 - val_loss: 3.0077
Epoch 3/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.0958 - loss: 3.0313 - val_accuracy: 0.2562 - val_loss: 2.6639
Epoch 4/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - accuracy: 0.1761 - loss: 2.7309 - val_accuracy: 0.3185 - val_loss: 2.2536
Epoch 5/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.2179 - loss: 2.4884 - val_accuracy: 0.3637 - val_loss: 2.0246
Epoch 6/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.2746 - loss: 2.2889 - val_accuracy: 0.4045 - val_loss: 1.9313
Epoch 7/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - accuracy: 0.3040 - loss: 2.1538 - val_accuracy: 0.4196 - val_loss: 1.7655
Epoch 8/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.3163 - loss: 2.0781 - val_accura

In [27]:
evaluate_model_plotly(model_resnet, x_test, y_test, model_name="ResNet50 Transfer Learning")
model_resnet.save('resnet50_emnist.keras')


📊 Calculating Metrics for ResNet50 Transfer Learning...

🔹 Classification Report [ResNet50 Transfer Learning]:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       120
           1       0.45      0.28      0.35       120
           2       0.32      0.57      0.41       120
           3       0.41      0.30      0.35       120
           4       0.50      0.60      0.55       120
           5       0.53      0.46      0.49       120
           6       0.47      0.30      0.37       120
           7       0.58      0.51      0.54       120
           8       0.53      0.75      0.62       120
           9       0.58      0.79      0.67       120
          10       0.53      0.37      0.43       120
          11       0.42      0.50      0.45       120
          12       0.52      0.41      0.46       120
          13       0.52      0.50      0.51       120
          14       0.45      0.34      0.39       120
          15       0.54 

# inception v1

In [30]:
from tensorflow.keras.applications import InceptionV3

log_dir = "logs/fit/inception_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

inputs = Input(shape=(32, 32, 1), name="img_input")

x = layers.Conv2D(3, (3, 3), padding='same', name='rgb_adapter')(inputs)

## the built in model crashes if images are smaller than 75x75.##
x = layers.Resizing(75, 75, interpolation="bilinear", name='resize_adapter')(x)

base_model = InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(75, 75, 3)
)
base_model.trainable = False ##freeze weights##

x = base_model(x, training=False)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(28, activation='softmax')(x)

model_inception = models.Model(inputs=inputs, outputs=outputs, name="InceptionV3_Transfer")

model_inception.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_inception.summary()

history_inception = model_inception.fit(
    x_train, y_train,
    epochs=15,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback]
)

# --- 8. EVALUATE & SAVE ---
# Use your Plotly evaluation function
evaluate_model_plotly(model_inception, x_test, y_test, model_name="InceptionV3 Transfer")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 26s 0us/step


Model: "InceptionV3_Transfer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ img_input (InputLayer)          │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rgb_adapter (Conv2D)            │ (None, 32, 32, 3)      │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resize_adapter (Resizing)       │ (None, 75, 75, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 1, 1, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 28)             │        14,364 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,866,266 (87.23 MB)

 Trainable params: 1,063,482 (4.06 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

🚀 Starting InceptionV3 Training...
Epoch 1/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 49s 123ms/step - accuracy: 0.3509 - loss: 2.2364 - val_accuracy: 0.6690 - val_loss: 1.0424
Epoch 2/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 63ms/step - accuracy: 0.6371 - loss: 1.1170 - val_accuracy: 0.7077 - val_loss: 0.8924
Epoch 3/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - accuracy: 0.7032 - loss: 0.8944 - val_accuracy: 0.7286 - val_loss: 0.8177
Epoch 4/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - accuracy: 0.7390 - loss: 0.7669 - val_accuracy: 0.7563 - val_loss: 0.7462
Epoch 5/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - accuracy: 0.7723 - loss: 0.6676 - val_accuracy: 0.7583 - val_loss: 0.7157
Epoch 6/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7955 - loss: 0.6077 - val_accuracy: 0.7634 - val_loss: 0.7027
Epoch 7/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 12s 58ms/step - accuracy: 0.8153 - loss: 0.5583 - val_accuracy: 0.7759 - val_loss: 0.6740
Epoch 8/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step 

In [ ]:
# 1. Force kill all running TensorBoard instances

%load_ext tensorboard
%tensorboard --logdir logs/fit --host 0.0.0.0 --port 6007

## mobile net

In [33]:
from tensorflow.keras.applications import MobileNetV2

log_dir = "logs/fit/mobilenet_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

inputs = Input(shape=(32, 32, 1), name="img_input")


x = layers.Conv2D(3, (3, 3), padding='same', name='rgb_adapter')(inputs)

base_model = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(32, 32, 3)
)
base_model.trainable = False # Freeze weights


x = base_model(x, training=False)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(28, activation='softmax')(x)

model_mobile = models.Model(inputs=inputs, outputs=outputs, name="MobileNetV2_Transfer")

model_mobile.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_mobile.summary()

history_mobile = model_mobile.fit(
    x_train, y_train,
    epochs=15,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback]
)

evaluate_model_plotly(model_mobile, x_test, y_test, model_name="MobileNetV2 Transfer")


/tmp/ipykernel_6042/4089816530.py:11: UserWarning:

`input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.



Model: "MobileNetV2_Transfer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ img_input (InputLayer)          │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rgb_adapter (Conv2D)            │ (None, 32, 32, 3)      │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1, 1, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 28)             │        14,364 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,928,250 (11.17 MB)

 Trainable params: 670,266 (2.56 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.1225 - loss: 3.1272 - val_accuracy: 0.2670 - val_loss: 2.5134
Epoch 2/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.2588 - loss: 2.4707 - val_accuracy: 0.3057 - val_loss: 2.2483
Epoch 3/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.2941 - loss: 2.2645 - val_accuracy: 0.3301 - val_loss: 2.1389
Epoch 4/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.3175 - loss: 2.1637 - val_accuracy: 0.3592 - val_loss: 2.0621
Epoch 5/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3351 - loss: 2.0826 - val_accuracy: 0.3714 - val_loss: 2.0142
Epoch 6/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.3558 - loss: 2.0416 - val_accuracy: 0.3798 - val_loss: 1.9721
Epoch 7/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.3594 - loss: 1.9827 - val_accuracy: 0.3926 - val_loss: 1.9425
Epoch 8/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.3727 - loss: 1.9616 - val_acc